In [1]:
import sqlite3
import pandas as pd

In [2]:
OUTPUT_DIR = "../../output/healthcare"

#### Load

In [3]:
data = {}
tables = ["patients", "doctors", "hospitals", "insurers", "admissions", "medications", "test_results"]

for t in tables:
    data[t] = pd.read_csv(f"{OUTPUT_DIR}/{t}.csv")

#### Transform

In [4]:
# Remove whitespace in column names
for t in tables:
    data[t].columns = data[t].columns.str.replace(' ', '')

#### Load

In [5]:
conn = sqlite3.connect(f"{OUTPUT_DIR}/healthcare.sqlite3")
cursor = conn.cursor()

# Enforce foreign keys
cursor.execute("PRAGMA foreign_keys = ON;")

cursor.executescript("""
DROP TABLE IF EXISTS TestResults;
DROP TABLE IF EXISTS Medications;
DROP TABLE IF EXISTS Admissions;
DROP TABLE IF EXISTS Patients;
DROP TABLE IF EXISTS Doctors;
DROP TABLE IF EXISTS Hospitals;
DROP TABLE IF EXISTS Insurers;

CREATE TABLE Patients (
    PatientID INTEGER PRIMARY KEY,
    Name TEXT,
    Age INTEGER,
    Gender TEXT,
    BloodType TEXT,
    MedicalCondition TEXT
);

CREATE TABLE Doctors (
    DoctorID INTEGER PRIMARY KEY,
    Doctor TEXT
);

CREATE TABLE Hospitals (
    HospitalID INTEGER PRIMARY KEY,
    Hospital TEXT
);

CREATE TABLE Insurers (
    InsuranceID INTEGER PRIMARY KEY,
    InsuranceProvider TEXT
);

CREATE TABLE Admissions (
    AdmissionID INTEGER PRIMARY KEY,
    PatientID INTEGER,
    DoctorID INTEGER,
    HospitalID INTEGER,
    InsuranceID INTEGER,
    DateofAdmission datetime,
    DischargeDate TEXT,
    RoomNumber TEXT,
    AdmissionType TEXT,
    BillingAmount REAL,
    FOREIGN KEY(PatientID) REFERENCES Patients(PatientID),
    FOREIGN KEY(DoctorID) REFERENCES Doctors(DoctorID),
    FOREIGN KEY(HospitalID) REFERENCES Hospitals(HospitalID),
    FOREIGN KEY(InsuranceID) REFERENCES Insurers(InsuranceID)
);

CREATE TABLE Medications (
    MedicationID INTEGER PRIMARY KEY,
    AdmissionID INTEGER,
    Medication TEXT,
    FOREIGN KEY(AdmissionID) REFERENCES Admissions(AdmissionID)
);

CREATE TABLE TestResults (
    TestResultID INTEGER PRIMARY KEY,
    AdmissionID INTEGER,
    TestResults TEXT,
    FOREIGN KEY(AdmissionID) REFERENCES Admissions(AdmissionID)
);
""")

data["patients"].to_sql('Patients', conn, if_exists='append', index=False)
data["doctors"].to_sql('Doctors', conn, if_exists='append', index=False)
data["hospitals"].to_sql('Hospitals', conn, if_exists='append', index=False)
data["insurers"].to_sql('Insurers', conn, if_exists='append', index=False)
data["admissions"].to_sql('Admissions', conn, if_exists='append', index=False)
data["medications"].to_sql('Medications', conn, if_exists='append', index=False)
data["test_results"].to_sql('TestResults', conn, if_exists='append', index=False)

54966

##### Testing

In [6]:
query = "SELECT * FROM Admissions LIMIT 5;"
print(pd.read_sql(query, conn))

   AdmissionID  PatientID  DoctorID  HospitalID  InsuranceID DateofAdmission  \
0            1          1         1           1            1      2024-01-31   
1            2          2         2           2            2      2019-08-20   
2            3          3         3           3            3      2022-09-22   
3            4          4         4           4            2      2020-11-18   
4            5          5         5           5            3      2022-09-19   

  DischargeDate RoomNumber AdmissionType  BillingAmount  
0    2024-02-02        328        Urgent   18856.281306  
1    2019-08-26        265     Emergency   33643.327287  
2    2022-10-07        205     Emergency   27955.096079  
3    2020-12-18        450      Elective   37909.782410  
4    2022-10-09        458        Urgent   14238.317814  


##### Ending the connection

In [7]:
conn.close()